#### **Setup and Imports**

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, color, feature, filters, exposure, measure, img_as_ubyte
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.transform import resize
import pywt
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

#### **Data Loading**

In [2]:
base_path = "E:/Rawan/Projects/Machine Learning/Project/Neurodegenerative-Diseases-Classifier/Data"
classes = ["alzheimer", "parkinson", "normal"]
class_paths = {cls: os.path.join(base_path, cls) for cls in classes}

# Function to load images
def load_image(image_path):
    # Load image
    img = io.imread(image_path)
    
    # Convert to grayscale if it's RGB
    if len(img.shape) > 2:
        img = color.rgb2gray(img)
    
    # Ensure the image is in the correct format for feature extraction
    img = img_as_ubyte(img)
    
    return img

#### **First-Order Statistical Features**

First-order statistical features capture the distribution of pixel intensities in the MRI image. These features are important because they can reveal overall brightness, contrast, and intensity variations that may indicate tissue abnormalities.

Key first-order features include:
- **Mean**: Average intensity, which can indicate overall brain density
- **Standard Deviation**: Variation in intensity, which can reflect tissue heterogeneity
- **Skewness**: Asymmetry of the intensity distribution
- **Kurtosis**: "Peakedness" of the intensity distribution
- **Entropy**: Measure of randomness or unpredictability in pixel values

These features are particularly useful for detecting general changes in brain tissue composition that occur in neurodegenerative diseases.

In [3]:
def extract_first_order_features(image):
    """Extract first-order statistical features from an image."""
    # Flatten the image to get a 1D array of pixel values
    pixels = image.flatten()
    
    # Calculate basic statistics
    mean = np.mean(pixels)
    std = np.std(pixels)
    
    # Calculate histogram
    hist, _ = np.histogram(pixels, bins=256, range=(0, 255))
    hist = hist / np.sum(hist)  # Normalize histogram
    
    # Calculate skewness
    skewness = np.sum(((pixels - mean) / std) ** 3) / len(pixels) if std > 0 else 0
    
    # Calculate kurtosis
    kurtosis = np.sum(((pixels - mean) / std) ** 4) / len(pixels) if std > 0 else 0
    
    # Calculate entropy
    entropy = -np.sum(hist * np.log2(hist + 1e-10))
    
    # Calculate percentiles
    p10 = np.percentile(pixels, 10)
    p25 = np.percentile(pixels, 25)
    p75 = np.percentile(pixels, 75)
    p90 = np.percentile(pixels, 90)
    
    # Calculate range
    min_val = np.min(pixels)
    max_val = np.max(pixels)
    range_val = max_val - min_val
    
    # Return as dictionary
    return {
        'mean': mean,
        'std': std,
        'skewness': skewness,
        'kurtosis': kurtosis,
        'entropy': entropy,
        'p10': p10,
        'p25': p25,
        'p75': p75,
        'p90': p90,
        'min': min_val,
        'max': max_val,
        'range': range_val
    }

#### **Gray Level Co-occurrence Matrix (GLCM) Features**

GLCM features are second-order statistical texture features that capture spatial relationships between pixels. These features are particularly valuable for neurodegenerative disease classification because they can detect subtle texture changes in brain tissue that may not be visible to the naked eye.

The GLCM computes how often pairs of pixels with specific values occur in a specific spatial relationship. From this matrix, we can derive several important properties:

- **Contrast**: Measures local variations in the GLCM
- **Correlation**: Measures joint probability occurrence of specified pixel pairs
- **Energy/ASM**: Provides the sum of squared elements in the GLCM
- **Homogeneity**: Measures the closeness of the distribution of elements in the GLCM to its diagonal

Research has shown that GLCM features can effectively differentiate between AD, PD, and normal brain tissues by capturing textural differences in regions like the hippocampus, thalamus, and cortical areas.

In [8]:
def extract_glcm_features(image, distances=[1, 3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """Extract GLCM texture features from an image."""
    # Make sure image is in uint8 format with values 0-255
    if image.dtype != np.uint8:
        image = img_as_ubyte(image)
    
    # Rescale to fewer gray levels (e.g., 0-15)
    bins = 16  # Using 16 gray levels
    max_val = np.max(image)
    
    # Ensure max value is less than bins
    if max_val >= bins:
        # Rescale image to 0-15
        img_quantized = np.uint8(np.floor(image.astype(float) * (bins-1) / 255.0))
    else:
        img_quantized = image
    
    # Calculate GLCM
    glcm = graycomatrix(img_quantized, distances=distances, angles=angles, 
                        levels=bins, symmetric=True, normed=True)
    
    # Calculate GLCM properties
    contrast = graycoprops(glcm, 'contrast').mean()
    dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    energy = graycoprops(glcm, 'energy').mean()
    correlation = graycoprops(glcm, 'correlation').mean()
    ASM = graycoprops(glcm, 'ASM').mean()
    
    # Return as dictionary
    return {
        'glcm_contrast': contrast,
        'glcm_dissimilarity': dissimilarity,
        'glcm_homogeneity': homogeneity,
        'glcm_energy': energy,
        'glcm_correlation': correlation,
        'glcm_ASM': ASM
    }

#### **Local Binary Pattern (LBP) Features**

Local Binary Pattern (LBP) is a powerful texture descriptor that characterizes the local structure of an image by comparing each pixel with its neighboring pixels. LBP is particularly useful for medical image analysis because:

1. It's computationally efficient
2. It's invariant to monotonic gray-level changes (which makes it robust to lighting variations)
3. It can capture micro-patterns in the image

For neurodegenerative disease classification, LBP can detect subtle texture changes in brain tissue that occur due to neurodegeneration. Studies have shown that LBP features can effectively differentiate between AD, PD, and normal brain tissues by capturing local texture patterns in regions like the hippocampus and cortical areas.

We'll extract LBP histograms from the images, which will serve as a compact representation of the texture patterns present in the brain MRI scans.

In [5]:
def extract_lbp_features(image, P=8, R=1):
    """Extract Local Binary Pattern features from an image."""
    # Calculate LBP
    lbp = local_binary_pattern(image, P, R, method='uniform')
    
    # Calculate LBP histogram
    n_bins = P + 2  # For uniform LBP
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    
    # Create feature dictionary
    lbp_features = {}
    for i, h in enumerate(hist):
        lbp_features[f'lbp_bin_{i}'] = h
    
    return lbp_features

#### **Wavelet Features**

Wavelet transform is a powerful technique for analyzing images at multiple scales and orientations. By decomposing an image into different frequency components, wavelets can capture both global and local texture information.

For neurodegenerative disease classification, wavelet features are particularly valuable because:

1. They can detect subtle changes in brain tissue texture at different scales
2. They can capture directional information (horizontal, vertical, and diagonal details)
3. They provide a compact representation of the image's texture

Research has shown that wavelet features can effectively differentiate between AD, PD, and normal brain tissues by capturing multi-scale texture information in regions like the hippocampus, cortical areas, and subcortical structures.

We'll use the Discrete Wavelet Transform (DWT) to decompose the image into approximation, horizontal, vertical, and diagonal coefficients, and then extract statistical features from these coefficients.

In [6]:
def extract_wavelet_features(image):
    """Extract wavelet features from an image."""
    # Apply wavelet transform
    coeffs = pywt.dwt2(image, 'db1')
    LL, (LH, HL, HH) = coeffs
    
    # Extract statistical features from each sub-band
    wavelet_features = {}
    
    for name, coeff in zip(['LL', 'LH', 'HL', 'HH'], [LL, LH, HL, HH]):
        # Calculate basic statistics
        mean = np.mean(coeff)
        std = np.std(coeff)
        energy = np.sum(coeff**2) / coeff.size
        entropy = -np.sum((coeff**2) * np.log2(coeff**2 + 1e-10)) / coeff.size
        
        wavelet_features[f'wavelet_{name}_mean'] = mean
        wavelet_features[f'wavelet_{name}_std'] = std
        wavelet_features[f'wavelet_{name}_energy'] = energy
        wavelet_features[f'wavelet_{name}_entropy'] = entropy
    
    return wavelet_features

#### **Edge and Gradient Features**

Edge and gradient features capture important structural information in brain MRI scans by highlighting boundaries between different tissue types and anatomical structures. These features are valuable for neurodegenerative disease classification because:

1. They can detect changes in brain structure boundaries that occur due to atrophy
2. They can highlight subtle changes in tissue interfaces that may not be visible in the original image
3. They provide information about the direction and magnitude of intensity changes

For diseases like Alzheimer's and Parkinson's, changes in brain structure boundaries (such as ventricle enlargement or cortical thinning) are important diagnostic indicators. Edge and gradient features can help quantify these changes.

We'll extract features like gradient magnitude statistics, Canny edge statistics, and Sobel filter responses to capture this structural information.

In [7]:
def extract_edge_gradient_features(image):
    """Extract edge and gradient features from an image."""
    # Calculate gradient using Sobel operator
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Calculate gradient magnitude and direction
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    direction = np.arctan2(sobely, sobelx)
    
    # Calculate Canny edges
    edges = feature.canny(image, sigma=1)
    
    # Extract features
    features = {
        'gradient_mean': np.mean(magnitude),
        'gradient_std': np.std(magnitude),
        'gradient_energy': np.sum(magnitude**2) / magnitude.size,
        'gradient_entropy': -np.sum((magnitude**2) * np.log2(magnitude**2 + 1e-10)) / magnitude.size,
        'direction_mean': np.mean(direction),
        'direction_std': np.std(direction),
        'edge_density': np.sum(edges) / edges.size
    }
    
    return features

#### **Gray Level Run Length Matrix (GLRLM) Features**

Gray Level Run Length Matrix (GLRLM) is a texture analysis method that examines runs of pixels with the same gray level value in a specific direction. A run is defined as a set of consecutive pixels with the same gray level in a specific direction.

GLRLM features are valuable for neurodegenerative disease classification because:

1. They can capture patterns of homogeneous regions in the brain
2. They provide information about the coarseness or fineness of texture
3. They can detect subtle changes in tissue texture that occur due to neurodegeneration

Studies have shown that GLRLM features can effectively differentiate between AD, PD, and normal brain tissues by capturing run-length based texture patterns in regions like the hippocampus, thalamus, and cortical areas.

We'll extract features like Short Run Emphasis (SRE), Long Run Emphasis (LRE), and Run Length Non-uniformity (RLN) to capture this texture information.

In [ ]:
def extract_glrlm_features(image):
    """Extract Gray Level Run Length Matrix features from an image without using mahotas."""
    # Quantize the image to fewer gray levels
    bins = 16
    img_quantized = np.digitize(image, np.linspace(0, 255, bins+1)) - 1
    
    # Define directions for run length calculation
    directions = [(1, 0),   # horizontal
                  (0, 1),   # vertical
                  (1, 1),   # diagonal
                  (1, -1)]  # anti-diagonal
    
    features = {}
    
    # Calculate run length features for each direction
    for idx, (dx, dy) in enumerate(directions):
        # Initialize counters
        short_runs = 0
        long_runs = 0
        total_runs = 0
        run_lengths = []
        
        # Process the image
        rows, cols = img_quantized.shape
        for i in range(rows):
            for j in range(cols):
                # Skip if we've already processed this pixel as part of a run
                if img_quantized[i, j] == -1:
                    continue
                
                # Start a new run
                run_val = img_quantized[i, j]
                run_length = 0
                
                # Follow the run in the current direction
                x, y = j, i
                while (0 <= x < cols and 0 <= y < rows and 
                       img_quantized[y, x] == run_val):
                    # Mark as processed
                    img_quantized[y, x] = -1
                    run_length += 1
                    
                    # Move to next pixel in direction
                    x += dx
                    y += dy
                
                # Record the run
                if run_length > 0:
                    run_lengths.append(run_length)
                    total_runs += 1
                    
                    # Classify as short or long run
                    if run_length <= 2:
                        short_runs += 1
                    if run_length >= 4:
                        long_runs += 1
        
        # Calculate features
        if total_runs > 0:
            # Short Run Emphasis (SRE)
            features[f'glrlm_sre_dir{idx}'] = short_runs / total_runs
            
            # Long Run Emphasis (LRE)
            features[f'glrlm_lre_dir{idx}'] = long_runs / total_runs
            
            # Run Length Non-uniformity (RLN)
            if len(run_lengths) > 0:
                features[f'glrlm_rln_dir{idx}'] = np.std(run_lengths) / np.mean(run_lengths) if np.mean(run_lengths) > 0 else 0
            else:
                features[f'glrlm_rln_dir{idx}'] = 0
        else:
            features[f'glrlm_sre_dir{idx}'] = 0
            features[f'glrlm_lre_dir{idx}'] = 0
            features[f'glrlm_rln_dir{idx}'] = 0
    
    # Calculate average features across all directions
    features['glrlm_sre'] = np.mean([features[f'glrlm_sre_dir{i}'] for i in range(4)])
    features['glrlm_lre'] = np.mean([features[f'glrlm_lre_dir{i}'] for i in range(4)])
    features['glrlm_rln'] = np.mean([features[f'glrlm_rln_dir{i}'] for i in range(4)])
    
    # Remove direction-specific features to keep the feature set smaller
    for i in range(4):
        del features[f'glrlm_sre_dir{i}']
        del features[f'glrlm_lre_dir{i}']
        del features[f'glrlm_rln_dir{i}']
    
    return features

#### **Feature Extraction Pipeline**

Now we'll combine all the feature extraction methods into a single pipeline. This pipeline will:

1. Load each image from the dataset
2. Extract all the features we've defined
3. Combine them into a single feature vector
4. Save the results to a CSV file for further analysis

In [10]:
def extract_all_features(image):
    """Extract all features from an image."""
    # Extract all features
    features = {}
    
    # First-order statistical features
    features.update(extract_first_order_features(image))
    
    # GLCM features
    features.update(extract_glcm_features(image))
    
    # LBP features
    features.update(extract_lbp_features(image))
    
    # Wavelet features
    features.update(extract_wavelet_features(image))
    
    # Edge and gradient features
    features.update(extract_edge_gradient_features(image))
    
    # GLRLM features
    features.update(extract_glrlm_features(image))
    
    return features

def process_dataset():
    """Process all images in the dataset and extract features."""
    # Create a list to store results
    results = []
    
    # Process each class
    for cls in classes:
        class_path = class_paths[cls]
        print(f"Processing {cls} images...")
        
        # Get all image files in the class folder
        image_files = [f for f in os.listdir(class_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        
        # Process each image
        for img_file in tqdm(image_files):
            img_path = os.path.join(class_path, img_file)
            
            try:
                # Load image
                image = load_image(img_path)
                
                # Extract features
                features = extract_all_features(image)
                
                # Add class label and image filename
                features['class'] = cls
                features['filename'] = img_file
                
                # Add to results
                results.append(features)
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Save to CSV
    df.to_csv('brain_mri_features.csv', index=False)
    print(f"Features extracted and saved to brain_mri_features.csv")
    
    return df

#### **Execution and Visualization**

Finally, let's execute the feature extraction pipeline and visualize some of the features.

In [ ]:
# Execute the feature extraction pipeline
if __name__ == "__main__":
    base_path = "E:/Rawan/Projects/Machine Learning/Project/Neurodegenerative-Diseases-Classifier/Data"
    classes = ["alzheimer", "parkinson", "normal"]

    # Process the dataset
    features_df = process_dataset()
    
    # Display the first few rows of the features DataFrame
    print(features_df.head())
    
    # Display basic statistics
    print("\nFeature statistics:")
    print(features_df.describe())
    
    # Count the number of samples per class
    print("\nClass distribution:")
    print(features_df['class'].value_counts())
    
    # Visualize some key features
    plt.figure(figsize=(15, 10))
    
    # Plot histograms of some first-order features by class
    for i, feature in enumerate(['mean', 'std', 'entropy']):
        plt.subplot(2, 3, i+1)
        for cls in classes:
            plt.hist(features_df[features_df['class'] == cls][feature], alpha=0.5, label=cls, bins=20)
        plt.title(f'Distribution of {feature}')
        plt.legend()
    
    # Plot histograms of some GLCM features by class
    for i, feature in enumerate(['glcm_contrast', 'glcm_homogeneity', 'glcm_energy']):
        plt.subplot(2, 3, i+4)
        for cls in classes:
            plt.hist(features_df[features_df['class'] == cls][feature], alpha=0.5, label=cls, bins=20)
        plt.title(f'Distribution of {feature}')
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('feature_distributions.png')
    plt.show()